# Auxiliary functions
This file contains all the algorithms used for cleaning the signal given and removing the noise. It also includes some auxiliar functions that are implemented in the code of the algorithms or that are used in the project in general.

## Imports

In [ ]:
!pip install netCDF4 numpy tqdm numba netCDF4

In [ ]:
import numba
from numba import jit, cuda
from netCDF4 import Dataset
import numpy as np
from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings(
    action="ignore", category=numba.NumbaPerformanceWarning, module="numba"
)

## Miscellaneous functions

### read_nc(nc_filename)
<a id="read_nc"></a>
This simple function just reads the data we want out of the files we were providen.

In [ ]:
def read_nc(nc_filename):
    # We use the Dataset class from the netCDF4 library to help us open and extract data from the nc file in an easy way
    rootgrp = Dataset(nc_filename, "r")
    
    latitude_list = rootgrp["science/grids/data/latitude"][:].data
    longitude_list = rootgrp["science/grids/data/longitude"][:].data
    
    # We return a tuple with a list of latitudes and a list of longitudes, but if we uncomment the lines below
    # we can instead (or also) return a tuple fo the form: (max_latitude, max_longitud, min_latitude, min_longitud),
    # which constitutes the corners of the "imgage"
    #image_corner_top_left = (np.max(latitude_list), np.min(longitude_list))
    #image_corner_bottom_right = (np.min(latitude_list), np.max(longitude_list))
    #image_corners = image_corner_top_left, image_corner_bottom_right
    
    unwrapped_phase = rootgrp["science/grids/data/unwrappedPhase"][:].data
    coherence = rootgrp["science/grids/data/coherence"][:].data
    amplitude = rootgrp["science/grids/data/amplitude"][:].data
    connected_components = rootgrp["science/grids/data/connectedComponents"][:].data
    
    wavelength = rootgrp["science/radarMetaData/wavelength"][:].data
    
    rootgrp.close()
    
    # all the variables returned are np.arrays
    return (latitude_list, longitude_list), unwrapped_phase, coherence, amplitude, connected_components, wavelength

### draw_phase_amplitude(unwrapped_phase, amplitude)
<a id="print_phase_amplitude"></a>
This functions draws the wrapped phase (–π, π) on top of the amplitude, with a colorbar as a legend, returns the figure so
we can print.

In [ ]:
from matplotlib import pyplot as plt

def draw_phase_amplitude(unw_phase, amp, size=(15, 15), alpha=0.5, ticks=True, title="Wrapped phase on top of amplitude"):
    w_phase = np.fmod(unw_phase, np.pi)
    fig, ax = plt.subplots()
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_title(title)

    ax.imshow(amp, cmap="gray", alpha=1)
    img = ax.imshow(w_phase, cmap="rainbow", alpha=alpha)
    
    if ticks:
        cbar = plt.colorbar(img, ax=ax, shrink=0.5, ticks=[3.13, 3.14/2, 0, -3.14/2, -3.13])
        cbar.ax.set_yticklabels(['π', 'π/2', '0', '-π/2', '-π'])
    else:
        plt.colorbar(img, ax=ax, shrink=0.5)
        
    fig.set_size_inches(size[0], size[1])
    
    return fig

### rudimentary_reshape(all_imgs)

This is the first functin we did to transfor all the images to the same size, it is also very crude and it doesn't work well if the difference between sizes is too big

In [ ]:
def rudimentary_reshape(all_imgs):
    # slicing the images, making them the same size (min size of all the images given)
    r = []
    c = []
    for img in all_imgs:
        r.append(img.shape[0]) # the height of the picture
        c.append(img.shape[1]) # the width of the picture

    return [img[:min(r), :min(c)] for img in coherence_imgs_arr]

### normalize(unwrapped_phase_mat)

In [ ]:
def normalize(unwrapped_phase_mat):
    vmin, vmax = np.min(unwrapped_phase_mat), np.max(unwrapped_phase_mat)
    return (unwrapped_phase_mat - vmin) / (vmax - vmin)

### reshape(list_of_images)

In this part of code we get all the latitudes and logitudes of the images and reshape all them to the same size in the intersection of all the images that we are reshaping. So that we can overlap images without problems.
It is divided in two parts: get_data(elements); and change_shape(input_).
The first part gets the list of nc files of the data we want and the latter makes the changes of the shape of the list of images in function of the elements we've got in get_data().

In [ ]:
'''
This function gets the elements Corners (used to reshape the different images to the same scale at the same point (latitude,longitude))
And the data of unwrapped_phase and coherence (the final output we have to reshape)
'''
# def get_data(ncdata):
#     unwrapped_phase_array = []
#     coherence_array = []
#     corner_data = []
#     for nc in ncdata:
#         lat_lon_lists, unwrapped_phase, coherence, _, _, _ = nc

#         unwrapped_phase_array.append(unwrapped_phase)
#         coherence_array.append(coherence)
#         corner_data.append((lat_lon_lists, unwrapped_phase, coherence))
        
#     return corner_data, unwrapped_phase_array, coherence_array

# def get_data(ncdata):
#     unwrapped_phase_array = []
#     coherence_array = []
#     corner_data = []
#     for nc in ncdata:
#         lat_lon_lists, unwrapped_phase, coherence, _, _, _ = nc

#         unwrapped_phase_array.append(unwrapped_phase)
#         coherence_array.append(coherence)
#         corner_data.append((lat_lon_lists, unwrapped_phase, coherence))
        
#     return corner_data, unwrapped_phase_array, coherence_array

'''
This one is the shape changer of the list of images in function of latitudes and longitudes of the data
'''
def reshape(input_data):
    # input_ = [(cordinates, uw_phase, coherence), ...] where cordinates = [([latitudes], [longitudes]), ...]
    all_lat = []
    all_long = []
    
    cordinates = []
    for cor, _, _, _, _, _ in input_data:
        cordinates.append(cor)
        
    for element in cordinates:
        all_lat.append(element[0]) #arrays of all the latitudes
        all_long.append(element[1]) #arrays of all the longitudes
        
    #max lats and max longs are the list of the touple (latitude, longitude) at the top right corner
    #same thing to min lats and min longs but in the bottom left corner
    max_lats = []
    min_lats = []
    max_longs = []
    min_longs = []
    for lat, long in zip(all_lat, all_long):
        max_lats.append(max(lat))
        min_lats.append(min(lat))
        max_longs.append(max(long))
        min_longs.append(min(long))
    
    #the intersection will be the least upperbound and greatest lowerbound of the latitudes and longitudes (the intersection)
    new_height = min(max_lats) - max(min_lats) 
    new_width = min(max_longs) - max(min_longs)
    
    resized_img_list = []
    for i, value in enumerate(input_data):
        latlon, unwrapped_phase, coherence, amplitude, cc, wl = value
        size_y, size_x = unwrapped_phase.shape                      
        
        #getting the proportional new sizes x and y
        new_y = int(size_y * new_height / (max_lats[i] - min_lats[i]))
        new_x = int(size_x * new_width / (max_longs[i] - min_longs[i]))
        
        resized_img_list.append((latlon, unwrapped_phase[:new_y, :new_x], coherence[:new_y, :new_x], amplitude[:new_y, :new_x], cc[:new_y, :new_x], wl))
    
    return resized_img_list

# def reshape(ncdata):
#     # data = get_data(ncdata)
#     # data = [(lat_lon_lists, unwrapped_phase, coherence, amplitude) for lat_lon_lists, unwrapped_phase, coherence, amplitude, _, _ in ncdata]
#     return change_shape(ncdata)

## Algorithms

### simple_convolution(array, resolution)
<a id="simple_convolution"></a>

The simple convolution function makes an average between a range of points in the image and "softens" it to eliminate some noise at the cost of loss of data.
It was the first function we did and it is very crude. The resolution of the image drops substantially

In [ ]:
def simple_convolution(arr, resolution):
    shapey, shapex = arr.shape # gets the x and y values of the shape
    new_shape = new_shapey, new_shapex = shapey-resolution*2, shapex-resolution*2 # the new shape is the shape minus the external arrays of the image
    new_arr = np.zeros(new_shape)

    for i in tqdm(range(resolution, new_shapey), desc="Simple convolution"):
        for j in range(resolution, new_shapex):
            new_arr[i-1, j-1] = arr[i-resolution:i+resolution+1, j-resolution:j+resolution+1].sum() # this makes the average between the "resolution" size matrixes 
            
    return new_arr

### convolution_with_kernel(array, kernel)
<a id="ConvKer"></a>

This was the second function of convolution we did, the difference is that there is the variable kernel, that is an array with the values for a ponderate convolution.

In [ ]:
def convolution_with_kernel(array, kernel):
    kernel_size = kernel.shape[0]
    ks = int(kernel_size / 2)
    new_image = np.zeros(shape=(array.shape[0] - (2 * ks), array.shape[1] - (2 * ks)))
    
    for y in range(ks, array.shape[0] - ks):
        for x in range(ks, array.shape[1] - ks):
            out = np.multiply(kernel, array[y - ks:y + ks+ 1, x - ks: x + ks+ 1]) # multiply the points by the kernel values and makes the ponderate average
            new_image[y- ks, x - ks] = np.average(out)
    
    return new_image 

### compute_exp_kernel(kernel_size, b)

It calculates the values of the kernel using the negative exponential function. This kernel uses the Frost's algorithm.

In [ ]:
def compute_exp_kernel(kernel_size, b):
    # Calculate kernel
    kernel = np.zeros(shape=(kernel_size, kernel_size))
    center = (kernel_size - 1) // 2
    center = np.array([1.0, 1.0]) * center
    for y in range(kernel_size):
        for x in range(kernel_size):
            dis = np.linalg.norm(center - [y, x])
            kernel[y, x] = 2.71828 ** (-b  * abs(dis))
    
    kernel = kernel / np.sum(kernel)
    return kernel

### convolution_with_exp(array, kernel_size, b)

In this function, the values of the kernel are ponderated by the exponential function

In [ ]:
def convolution_with_exp(array, kernel_size, b):
    # kernel_size must always be odd
    kernel = compute_exp_kernel(kernel_size, b)
    return convolution_with_kernel(array, kernel)

### convolution_in_steroids(image, kernel, new_image)

A faster implementation of the convolution function written as a cuda kernel, allowing it to running in our nvidia gpus. The compilation to a cuda kernel is done by the Numba library.

In [ ]:
@cuda.jit(fastmath=True)
def convolution_in_steroids(image, kernel, new_image):
    # Get global positions inside the cuda grid space
    pos = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    
    # Compute some variables
    total = cuda.blockDim.x * cuda.gridDim.x 
    half_kernel = kernel.shape[0] // 2 
    new_image_size = new_image.shape[0] * new_image.shape[1]
    
    # Convolve in each pixel in parallel
    for active_pos in range(pos, new_image_size, total):
        # Local position on the image 
        new_x, new_y = active_pos % new_image.shape[1], active_pos // new_image.shape[1]
        old_x, old_y = new_x + half_kernel, new_y + half_kernel 
        v = 0

        # Compute the product of the kernel with the image 
        for i in range(kernel.shape[0] ** 2):
            kx, ky = i % kernel.shape[1], i // kernel.shape[1]
            v += kernel[ky, kx] * image[old_y + ky - half_kernel, old_x + kx - half_kernel]

        new_image[new_y % new_image.shape[0], new_x % new_image.shape[1]] = v

### convolution_in_exp_steroids(image, kernel_size, b)

This function just uses the above function to simplify the process. First computes the exponential kernel and then uses the fast convolution function.

In [ ]:
def convolution_in_exp_steroids(image, kernel_size, b, bpg=1024, tpb=512):
    # kernel size must always be odd
    kernel = compute_exp_kernel(kernel_size, b)
    k2 = 2 * (kernel_size // 2)
    new_image = np.zeros(shape=(image.shape[0] - k2, image.shape[1] - k2), dtype=np.float32)
    
    convolution_in_steroids[bpg, tpb](image, kernel, new_image)
    
    return new_image

### coherentise(phase_arr, coherence_arr, kernel_size, b)
<a id="coherentise"></a>

This function makes an average of a convolution with a very localized exponential ponderated by the coherence image.

In [ ]:
def coherentise(phase_arr, coherence_arr, kernel_size=15, b=10):
    # kernel_size must always be odd
    coherence_mat = normalize(coherence_arr * (coherence_arr > 0.1))
    return convolution_in_exp_steroids(np.multiply(normalize(phase_arr),coherence_mat),kernel_size,b)

### average_through_time(all_imgs)
This function reduces the noise by making a new array of the averages of the same points in different images of the same site through the time

In [ ]:
def average_through_time(all_imgs):
    output_img = np.zeros_like(all_imgs[0])
    for i in tqdm(range(output_img.shape[0]), desc='Average'):
        for j in range(output_img.shape[1]):
            # This array stores the value of the point i,j of all the images so then
            # we can add them and calculate the average of all the values(dividing by the number of imgaes)
            point = [img[i][j] for img in all_imgs]
            average = sum(point)/len(all_imgs)
            output_img[i][j] = average 
            
    return output_img

### average_with_convolution(phase_imgs, coherence_imgs)

In [ ]:
def average_with_convolution(phase_imgs, coherence_imgs, kernel_size=15, b=10):
    new_imgs = [coherentise(phase, coherence, kernel_size, b) for phase, coherence in zip(phase_imgs, coherence_imgs)]  
    return average_through_time(new_imgs)